In [2]:
# 라이브러리 불러오기
import spacy
import pandas as pd
import re
from spacy.lang.en import STOP_WORDS
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
class myconfig():
    jp_data_path = r"C:\Users\user\Desktop\토픽 모델링\2. 전처리\data\1.DataFrame_JP_all_clean.xlsx"
    kr_data_path = r"C:\Users\user\Desktop\토픽 모델링\2. 전처리\data\1.DataFrame_KR_all_clean.xlsx"
    stop_words_path = r"C:\Users\user\Desktop\토픽 모델링\3. LDA\data\불용어리스트.xlsx"

    all_data_path = r"C:\Users\user\Desktop\토픽 모델링\3. LDA\data\3.DataFrame_명사추가.xlsx"
    
    freq_data_kr_path = r"C:\Users\user\Desktop\토픽 모델링\3. LDA\data\3.vocab_freq_kr.xlsx"
    freq_data_jp_path = r"C:\Users\user\Desktop\토픽 모델링\3. LDA\data\3.vocab_freq_jp.xlsx"
       
    for_wc_kr_path = r"C:\Users\user\Desktop\토픽 모델링\3. LDA\data\3.wc용_data_kr.xlsx"
    for_wc__jp_path = r"C:\Users\user\Desktop\토픽 모델링\3. LDA\data\3.wc용_data_jp.xlsx"
    

mcfg = myconfig()

In [4]:
# 토크나이저 정의
nlp = spacy.load('en_core_web_sm')

In [5]:
# 불용어에 추가할 단어
custom_stop_words = list(pd.read_excel(mcfg.stop_words_path)["stopwords"])

# 기존 불용어에 사용자 정의 불용어 추가
for word in custom_stop_words:
    nlp.vocab[word].is_stop = True

# 다시 모델 로드 (불용어가 추가된 상태)
# nlp = spacy.load("en_core_web_sm")

In [6]:
# 데이터 불러오기
df_jp = pd.read_excel(mcfg.jp_data_path)
df_kr = pd.read_excel(mcfg.kr_data_path)

In [7]:
len(df_jp)

239

In [8]:
len(df_kr)

1237

In [9]:
df_jp

,Title,Publised,Journal,Abstract
0,生涯スポーツ「ミニテニス」の効用,2011,日本橋学館大学紀要,This study focuses on mini tennis a regional s...
1,転倒経験と生涯スポーツとの関係,2009,理学療法学Supplement,The purpose of this study is to investigate th...
2,生涯スポーツと高校体育,2016,日本体育学会大会予稿集,The theme of this study was made possible by c...
3,生涯スポーツ（論）と学校体育,2016,日本体育学会大会予稿集,The purpose of this two year symposium is to e...
4,生涯スポーツと運動部活動,2016,日本体育学会大会予稿集,Lifelong sports have conceptual dimensions tha...
...,...,...,...,...
234,ライフスタイル・アプローチによるスポーツニーズの発掘に関する研究,2012,生涯スポーツ学研究,The purpose of this study is to explore sports...
235,地域スポーツクラブにみる営利・非営利の法人選択の一考察(大会報告論文:異領域で対話可能なシス...,2008,社会・経済システム,The market of social human service has been co...
236,東日本大震災で被災したスポーツ集団の復興プロセス,2012,スポーツ社会学研究,The Great East Japan Earthquake had a signific...
237,スポーツ欲求から公共性へ,2011,スポーツ社会学研究,According to the Cabinet Office s New Public d...


In [10]:
# 데이터 concat
df = pd.concat([df_jp,df_kr],ignore_index=True)
len(df)

1476

In [11]:
# "Abstract" 만 따로 추출
df_all = df.copy()
df_all_abstract = pd.DataFrame(df_all["Abstract"], columns=["Abstract"])
len(df_all_abstract)

1476

In [12]:
# 명사 추출 및 단.복수 통일 / 불용어 버리기
def extract_nouns_and_lemmas(text):   
    doc = nlp(text)
    nouns_and_lemmas = []
    
    for token in doc:
        if token.pos_ == 'NOUN':
            token = token.lemma_
            if nlp.vocab[token].is_stop == False:
                if len(token) >= 3:
                    nouns_and_lemmas.append(token)

    return nouns_and_lemmas

In [13]:
abstract_noun = []

for text in df_all_abstract["Abstract"]:
    nouns = extract_nouns_and_lemmas(text)
    abstract_noun.append(nouns)

len(abstract_noun)

1476

In [17]:
df_all["Abstract_token"] = abstract_noun

In [18]:
# 빈도 저장
df_all.to_excel(mcfg.all_data_path)

## 단어 별 빈도 확인

In [31]:
# 추출 단어 확인(5개 이상)
cv = CountVectorizer(tokenizer=extract_nouns_and_lemmas , min_df=5)
dtm = cv.fit_transform(df_all["Abstract"][239:])

c:\Users\user\anaconda3\envs\PY39_\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [32]:
# 단어 목록
cv.get_feature_names_out()

array(['ability', 'absence', 'acceptance', 'access', 'accessibility',
       'accident', 'accomplishment', 'accordance', 'achievement',
       'acquisition', 'act', 'action', 'activation', 'activity',
       'actualization', 'adaptation', 'addiction', 'addition',
       'adherence', 'adjustment', 'administration', 'adolescent', 'adult',
       'advertisement', 'advertising', 'advice', 'aerobics', 'affair',
       'affect', 'affection', 'age', 'aging', 'ahp', 'aid', 'aim',
       'alternative', 'anger', 'answer', 'anxiety', 'appearance',
       'application', 'approach', 'appropriateness', 'area', 'art',
       'article', 'aspect', 'assessment', 'assignment', 'assistance',
       'assistant', 'association', 'assumption', 'athlete', 'athletic',
       'atmosphere', 'attachment', 'attention', 'attitude', 'attribute',
       'author', 'authority', 'autonomy', 'awareness', 'background',
       'badminton', 'balance', 'ball', 'barrier', 'base', 'baseball',
       'basis', 'basketball', 'beha

In [34]:
# 단어 빈도 데이터 프레임
word_count = pd.DataFrame({
    'word': cv.get_feature_names_out(),
    'count': dtm.sum(axis=0).flat
})

word_count_desc = word_count.sort_values('count', ascending=False)

In [35]:
# 빈도 저장
word_count_desc.to_excel(mcfg.freq_data_kr_path)

In [36]:
# 데이터프레임을 튜플 리스트로 변환
word_list = list(zip(word_count_desc['word'], word_count_desc['count']))

In [37]:
# 토크나이징한 최종 토큰 수 산출
all_words = []

for row in df_all["Abstract_token"][239:]:
    all_words.extend(row)

all_words_df = pd.DataFrame(all_words, columns=["words"])

In [38]:
# 모든 단어 저장(중복 허용_워드클라우드용)
all_words_df.to_excel(mcfg.for_wc_kr_path)